<a href="https://colab.research.google.com/github/mnagaku/digraj202103/blob/master/gamemech-analysis.ipynb" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"></a>

# ゲームメカニクス大全の分析


## 環境準備



パターンの関連を手打ちでdotにしたgamemech.dotを読み込む準備をする。

In [1]:
!sudo apt-get update
!sudo apt-get install -y graphviz libgraphviz-dev

Ign:1 http://repo.mongodb.org/apt/debian stretch/mongodb-org/3.6 InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Hit:4 http://packages.cloud.google.com/apt cloud-sdk InRelease                 
Hit:5 http://repo.mongodb.org/apt/debian stretch/mongodb-org/3.6 Release       
Hit:6 http://archive.ubuntu.com/ubuntu focal InRelease                         
Get:7 http://security.ubuntu.com/ubuntu focal-security InRelease [109 kB]      
Get:9 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]        
Hit:2 https://packages.cloud.google.com/apt kubernetes-xenial InRelease        
Get:10 http://archive.ubuntu.com/ubuntu focal-backports InRelease [101 kB]     
Fetched 327 kB in 2s (146 kB/s)    
Reading package lists... Done
Reading package lists... Done
Building dependency tree       
Reading state information... Done
graphviz is already the newest version (2.42.2-3build2).
libgraphviz-dev is already the newest version (2.42.2-3build2

In [2]:
!pip install pygraphviz networkx

In [3]:
pip install -U pygraphviz networkx

Requirement already up-to-date: pygraphviz in /opt/conda/lib/python3.8/site-packages (1.6)
Requirement already up-to-date: networkx in /opt/conda/lib/python3.8/site-packages (2.5)
Note: you may need to restart the kernel to use updated packages.


## gamemech.dotの読み込み

書籍に基づいてパターンの関係をgamemech.dotに書き起こした。

In [4]:
res = ! ! ls gamemech.dot
if "No such file or directory" in res[0]:
    !wget https://raw.githubusercontent.com/mnagaku/digraj202103/main/gamemech.dot
else:
    print("既にあるのでgithubからダウンロードしません")

既にあるのでgithubからダウンロードしません


In [5]:
!cat gamemech.dot

これを分析・加工のために読み込む。

In [6]:
from pygraphviz import *
g = AGraph("gamemech.dot")

## 隣接行列の折り返しと上三角化（正規化）

書き記されたパターンの関係は、関係の両端である2つのパターンの両方に記されているのが理想である。一方のみの記載では、記載のない方からの参照が行えないからである。本書のパターンの関係は、ほぼ一方のみの記載となっており、参照性に問題があると言える。

ただ、データとして扱う上では、無向グラフを隣接行列表現する場合、対角線を挟んだ上三角部と下三角部で同じ情報を二重に持つことになるため、ここではgamemech.dotの隣接情報を上三角部に集約する。

In [7]:
nodes = g.nodes()
edges = g.edges()

print("[参考]参照が双方向のものを抽出")
for edge in edges:
    if nodes.index(edge[0]) > nodes.index(edge[1]):
        g.delete_edge(edge[0], edge[1])
        if not (edge[1], edge[0]) in edges:
            g.add_edge(edge[1], edge[0])
        else:
            print("("+edge[1]+","+edge[0]+")") # 参照が双方向のものを抽出

g.write("gamemech2.dot")

!dot -Tpng gamemech2.dot -ogamemech2.png

[参考]参照が双方向のものを抽出
(str03,str07)
(trn01,trn04)
(trn07,trn08)
(trn09,trn10)
(trn14,trn16)
(trn10,act08)
(act02,act08)
(res01,res02)
(res07,res08)
(vic01,vic02)
(vic17,vic18)
(vic15,vic19)
(str03,unc04)
(str07,unc04)
(vic08,unc04)
(unc01,unc04)
(eco10,eco11)
(trn09,eco18)
(eco02,eco18)
(vic04,auc08)
(eco17,auc08)
(auc11,auc15)
(auc11,auc16)
(act10,mov12)
(res12,arc02)
(auc11,arc02)
(vic14,set04)
(unc03,car01)
(act02,car06)
(set05,car06)


関係の参照が双方のものは上記出力の通り。

gamemech2.dotに正規化した内容を出力する。

## 分類間の関係の検討

本検討では、同一分類（同一Chapter）内の全パターンを1つのノードにして、13のノード間の関係を調べる。2つの分類の間で、パターンの関係が何本あったかを数え上げ、2つの分類の間での全てのパターンの組み合わせ（直積）の数に占める割合を「重み」として使い、中心性指標を計算した。


In [8]:
import math
from pprint import pprint

chapters = {}
for node in g.nodes():
    if node[0:3] in chapters:
        chapters[node[0:3]] += 1
    else:
        chapters[node[0:3]] = 1

chapter_edges = {}
for n1 in range(0,len(chapters)-1):
    n1str = list(chapters)[n1]
    for n2 in range(n1+1,len(chapters)):
        n2str = list(chapters)[n2]
        if not n1str in chapter_edges:
            chapter_edges[n1str] = {}
        chapter_edges[n1str].update({n2str:0})

for edge in g.edges():
    if edge[0][0:3] != edge[1][0:3]:
        chapter_edges[edge[0][0:3]][edge[1][0:3]] += 1

chapg = AGraph(chapter_edges, splines=True, layout='fdp')

max = 0
min = 10000
for n1 in range(0,len(chapters)-1):
    n1str = list(chapters)[n1]
    n1color = [((int(n1/6)+2)%3)*((int(n1/6)+2)%3+1)*51/2+25, (int(n1/2)%3)*(int(n1/2)%3+1)*51/2+25, (n1%2)*128+64]
    for n2 in range(n1+1,len(chapters)):
        n2str = list(chapters)[n2]
        n2color = [((int(n2/6)+2)%3)*((int(n2/6)+2)%3+1)*51/2+25, (int(n2/2)%3)*(int(n2/2)%3+1)*51/2+25, (n2%2)*128+64]
        if chapter_edges[n1str][n2str] == 0:
            chapg.delete_edge(n1str,n2str)
        else:
            w = chapter_edges[n1str][n2str]/(chapters[n1str]*chapters[n2str])
            chapg.get_edge(n1str,n2str).attr['weight'] = w
            chapg.get_edge(n1str,n2str).attr['penwidth'] = math.sqrt(w)*50
            chapg.get_edge(n1str,n2str).attr['color'] = "#"+format(int((n1color[0]+n2color[0])/2), 'x')\
                +format(int((n1color[1]+n2color[1])/2), 'x')+format(int((n1color[2]+n2color[2])/2), 'x')\
                +format(int(255-w*2500), 'x')
            if w < min:
                min = w
            if w > max:
                max = w
                
print("weight max = "+str(max))
print("weight min = "+str(min))

chapg.write("chapg.dot")

!dot -Tpng chapg.dot -ochapg.png

weight max = 0.07575757575757576
weight min = 0.0024509803921568627


In [9]:
!cat chapg.dot

strict graph "" {
	graph [layout=fdp,
		splines=True
	];
	str -- trn	[color="#b21980f0",
		penwidth=3.8348249442368525,
		weight=0.0058823529411764705];
	str -- act	[color="#b23240e3",
		penwidth=5.270462766947299,
		weight=0.011111111111111112];
	str -- res	[color="#b23280e8",
		penwidth=4.767312946227961,
		weight=0.00909090909090909];
	str -- vic	[color="#b26540c0",
		penwidth=7.905694150420948,
		weight=0.025];
	str -- unc	[color="#b265805f",
		penwidth=12.613124477737825,
		weight=0.06363636363636363];
	str -- car	[color="#7f1940ab",
		penwidth=9.128709291752768,
		weight=0.03333333333333333];
	trn -- act	[color="#b23280ad",
		penwidth=9.03876907577734,
		weight=0.032679738562091505];
	trn -- res	[color="#b232c0e4",
		penwidth=5.1708768999501915,
		weight=0.0106951871657754];
	trn -- vic	[color="#b26580e1",
		penwidth=5.423261445466404,
		weight=0.011764705882352941];
	trn -- unc	[color="#b265c0e4",
		penwidth=5.1708768999501915,
		weight=0.0106951871657754];
	trn -- eco	[color="#

中心性指標HITSの算出

In [10]:
import networkx as nx
from pprint import pprint

ng = nx.Graph(chapg)
h,a = nx.hits_numpy(ng)
ht_sorted = sorted(h.items(), key=lambda x:x[1], reverse=True)
pprint(ht_sorted)

[('car', 0.1548781082204156),
 ('unc', 0.1152073205891176),
 ('set', 0.10889268883664407),
 ('eco', 0.07939159202678295),
 ('str', 0.07242228144315528),
 ('trn', 0.07095123263369472),
 ('wpl', 0.07048551141867586),
 ('act', 0.07010266644837405),
 ('arc', 0.06971028717124171),
 ('auc', 0.06643510879373393),
 ('vic', 0.047343748199008294),
 ('res', 0.04240464128857761),
 ('mov', 0.03177481293057843)]


中心性指標PageRankの算出

In [11]:
pr = nx.pagerank_numpy(ng)
pr_sorted = sorted(pr.items(), key=lambda x:x[1], reverse=True)
pprint(pr_sorted)

[('car', 0.14717689981213034),
 ('unc', 0.10574034758128491),
 ('set', 0.10280399242424931),
 ('arc', 0.08186366550113403),
 ('act', 0.07658226208542046),
 ('wpl', 0.07430049538869186),
 ('trn', 0.07231459272072795),
 ('eco', 0.06837950882918763),
 ('str', 0.06430314482198941),
 ('auc', 0.05892932419377202),
 ('vic', 0.053833846584474696),
 ('res', 0.05249319375153227),
 ('mov', 0.041278726305405265)]


CARに対する関係が非常に多いため、CARを中心としたネットワークである。

各分類の意味を考えた場合、VICが軸になることが予想されたが、実際にそれを見ることはできなかった。ただ、CARと関係を持つ分類が偏っているためか、ばねモデルによるレイアウトでネットワーク図を描いたものでは、中心性指標においては中位のTRNが中心となっている。但し、重みを一律数倍すると、レイアウトは大きく変化するため、ばねモデルによるレイアウトが安定しているとは言えず、レイアウトに意味を求めるのは適切とは言えない。

## 同一分類内の関係の洗い出し

同一分類内に見られる関係においては、そのいくつかが継承（汎化）関係であることが予想される。継承関係であれば、親が持つ形質は子に受け継がれるため、親が持つ関係を子も持つと考えられる。従って、継承関係を見い出すことで、形式的に関係を増やすことが可能となる。また、親子関係で結ばれたパターンを族としてのグループ化して扱うことも考えられる。

ここでは、同一分類内の関係を洗い出し、いくつかの分類について、その関係が継承を表しているか検討を行う。

In [12]:
label = ""
for edge in g.edges():
    if label != edge[0][0:3]:
        label = edge[0][0:3]
        print("\n"+label)
    if edge[0].startswith(edge[1][0:3]):
        print(edge)


str
('str02', 'str04')
('str02', 'str05')
('str02', 'str07')
('str03', 'str07')

trn
('trn01', 'trn04')
('trn01', 'trn05')
('trn02', 'trn05')
('trn02', 'trn15')
('trn05', 'trn06')
('trn07', 'trn08')
('trn07', 'trn12')
('trn07', 'trn14')
('trn09', 'trn10')
('trn10', 'trn16')
('trn12', 'trn14')
('trn14', 'trn16')
('trn16', 'trn17')

act
('act01', 'act04')
('act01', 'act10')
('act02', 'act08')
('act02', 'act11')
('act03', 'act11')
('act03', 'act13')
('act04', 'act17')
('act05', 'act11')
('act06', 'act07')
('act06', 'act09')
('act07', 'act08')
('act10', 'act11')
('act12', 'act14')
('act13', 'act14')
('act15', 'act16')
('act15', 'act18')

res
('res01', 'res02')
('res01', 'res05')
('res01', 'res07')
('res01', 'res14')
('res01', 'res18')
('res02', 'res05')
('res02', 'res07')
('res02', 'res13')
('res02', 'res21')
('res03', 'res04')
('res07', 'res08')
('res13', 'res14')
('res18', 'res20')

vic
('vic01', 'vic02')
('vic01', 'vic07')
('vic01', 'vic12')
('vic02', 'vic11')
('vic04', 'vic18')
('vic0

例えば、VICについては以下の16の関係が存在する。
```
('vic01', 'vic02')
('vic01', 'vic07')
('vic01', 'vic12')
('vic02', 'vic11')
('vic04', 'vic18')
('vic05', 'vic06')
('vic05', 'vic09')
('vic05', 'vic15')
('vic06', 'vic11')
('vic07', 'vic11')
('vic07', 'vic14')
('vic10', 'vic11')
('vic11', 'vic19')
('vic14', 'vic16')
('vic15', 'vic19')
('vic17', 'vic18')
```
これを親子関係でないか検討していく。

('vic01', 'vic02')：vic02はプレイヤアクション起点であり、vic01もプレイヤアクション起点含むが、vic02はアクションがそのまま得点、vic01は状況が解釈され得点、であるため、を親子関係や依存関係はない

('vic01', 'vic07')：vic07に「レースのように見えて実際にはそうではない」ものとして例があり、それは「vic01に相当」とされているため、相反関係である

('vic01', 'vic12')：「ハイソサエティ」の事例でvic01と無関係にvic12のみの適用が言及されており、親子関係や依存関係はない

('vic02', 'vic11')：vic02を得点システムとし、vic06+vic11を終了システムとする事例は多いが、必須とまでは言えない。それぞれ別の部分のメカニクスであるため、親子関係や依存関係はない

('vic04', 'vic18')：vic04における雪だるま式をvic18の導入で抑制できるが、必須とまでは言えない。親子関係や依存関係はない

('vic05', 'vic06')：ゲーム進行のそれぞれの局面で得点状況を把握できないとストレスだが、把握できすぎても終盤のプレイが最適化されすぎ、更に終了時ボーナスでの得点調整の要素も絡むという議論。直接的に2つのメカニクスの関係を論じていない

('vic05', 'vic09')
('vic05', 'vic15')
('vic06', 'vic11')

('vic07', 'vic11')：vic11の詳細化されたものの1つがvic07と考えられるため、vic11を親とする親子関係と考えられる

('vic07', 'vic14')
('vic10', 'vic11')
('vic11', 'vic19')
('vic14', 'vic16')
('vic15', 'vic19')
('vic17', 'vic18')



## 次数

In [13]:
import networkx as nx
from pprint import pprint

ng = nx.Graph(g)

dg = nx.degree(ng)
dg_sorted = sorted([(d, n) for n, d in dg], reverse=True)
pprint(dg_sorted)

[(18, 'unc01'),
 (15, 'arc02'),
 (11, 'act02'),
 (10, 'unc04'),
 (10, 'car06'),
 (10, 'auc04'),
 (9, 'str02'),
 (9, 'car05'),
 (8, 'trn10'),
 (8, 'mov02'),
 (8, 'eco05'),
 (8, 'car01'),
 (8, 'auc08'),
 (7, 'wpl08'),
 (7, 'wpl05'),
 (7, 'vic18'),
 (7, 'vic01'),
 (7, 'set04'),
 (7, 'res18'),
 (7, 'eco18'),
 (7, 'eco11'),
 (7, 'auc06'),
 (7, 'auc03'),
 (7, 'act15'),
 (6, 'vic11'),
 (6, 'vic07'),
 (6, 'trn05'),
 (6, 'res02'),
 (6, 'res01'),
 (6, 'auc11'),
 (6, 'arc06'),
 (6, 'act08'),
 (6, 'act06'),
 (6, 'act01'),
 (5, 'wpl06'),
 (5, 'vic14'),
 (5, 'vic06'),
 (5, 'unc02'),
 (5, 'trn16'),
 (5, 'trn09'),
 (5, 'trn07'),
 (5, 'trn02'),
 (5, 'str03'),
 (5, 'set02'),
 (5, 'eco13'),
 (5, 'eco02'),
 (5, 'auc13'),
 (5, 'act03'),
 (4, 'unc10'),
 (4, 'str07'),
 (4, 'str05'),
 (4, 'set05'),
 (4, 'res14'),
 (4, 'res07'),
 (4, 'mov04'),
 (4, 'mov01'),
 (4, 'eco16'),
 (4, 'eco14'),
 (4, 'eco01'),
 (4, 'car04'),
 (4, 'auc09'),
 (4, 'auc07'),
 (4, 'auc05'),
 (4, 'auc02'),
 (4, 'arc08'),
 (4, 'act16'),
 (4,

## pagerank

In [14]:
pr = nx.pagerank(ng)
pr_sorted = sorted(pr.items(), key=lambda x:x[1], reverse=True)
pprint(pr_sorted)

[('unc01', 0.02340117260150655),
 ('arc02', 0.020844095505140128),
 ('eco05', 0.015388844735507926),
 ('act02', 0.01476097498977991),
 ('str02', 0.013799148767806406),
 ('car05', 0.013789292236458616),
 ('unc04', 0.013650861466836453),
 ('car06', 0.013639434291487925),
 ('mov02', 0.013067279892646916),
 ('auc04', 0.012770412154343816),
 ('auc08', 0.011776250841981948),
 ('eco11', 0.011577618563098493),
 ('act15', 0.011396060903393603),
 ('vic18', 0.011180077588921405),
 ('car01', 0.011100201417809952),
 ('wpl05', 0.011028895276045421),
 ('set04', 0.010645350896962603),
 ('trn10', 0.010412737415670139),
 ('vic01', 0.010374233936784872),
 ('wpl08', 0.01012896339978216),
 ('vic11', 0.010074762762540157),
 ('auc06', 0.009918781831078215),
 ('trn07', 0.009398492439256329),
 ('eco18', 0.009297289761807302),
 ('act06', 0.009280114418063605),
 ('res18', 0.009268997500335378),
 ('auc03', 0.009201065982372988),
 ('vic07', 0.009131510682552146),
 ('act01', 0.009108553667508122),
 ('res02', 0.0089

## hits

In [15]:
(ht, dummy) = nx.hits(ng)
ht_sorted = sorted(ht.items(), key=lambda x:x[1], reverse=True)
pprint(ht_sorted)

[('unc01', 0.04343290701795568),
 ('act02', 0.028616178481053814),
 ('trn10', 0.02551356901275252),
 ('arc02', 0.02388474712074906),
 ('car06', 0.021982427835356692),
 ('auc04', 0.021470682959568617),
 ('car01', 0.020357005037333807),
 ('unc04', 0.020121313093619725),
 ('trn09', 0.017028037484564166),
 ('res18', 0.016396520375309996),
 ('eco18', 0.01545296248386207),
 ('res01', 0.015217775107350031),
 ('eco16', 0.015133757413383622),
 ('wpl08', 0.01453850731165331),
 ('auc03', 0.014143787921054578),
 ('res14', 0.014004893568045164),
 ('auc13', 0.013835588998984528),
 ('auc11', 0.01350736221455511),
 ('trn05', 0.013303622923629603),
 ('auc05', 0.013008703176993498),
 ('auc06', 0.012941087699963646),
 ('auc07', 0.012883997366095268),
 ('act08', 0.012587272347650637),
 ('vic01', 0.012390726941699862),
 ('vic07', 0.012093080214990615),
 ('unc08', 0.011682767629749851),
 ('res08', 0.01142732642449925),
 ('wpl06', 0.011405055526570597),
 ('wpl05', 0.01123828061614407),
 ('unc02', 0.011049482

res03、04の値的には、pagerankは微妙。ただ、hitsもエッジ本数稼いだのが強く出過ぎてる気がする。パラメータで調整かけるか、自分で固有ベクトル求める系計算するか。